# Experiment A: Bound Adherence Under Nonstationary Conditions
*Question: does empirical average regret stay within the $\gamma$-regret bounds?*

### Experiment Design
Grid search over the streams (stationary, slow, abrupt, periodic), lambda levels, and two deletion ratio regimes.

Use a static comparator and calibrated learning rate schedule.
Set gamma per-theory at calibration and fix the total horizon (ie. $T=50000$).

### Primary Analysis
For each of the experimental cells, evaluate the final $R_{T}/T$ to see whether the guarantee was met.
Multiple seeds per cell in order to make some kind of causal analysis on the results.

### Success Criteria
95% of the cells meet their guarantees then I would call that a success. Which cells experience more successes than others? Can I make a heatmap that shows the regret results per cell?

## Import Statements and Versioning

This uses pretty standard library imports, but the torch requirement can stress the memory limits of a host.

In [1]:
# set global seed
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

def set_seed(seed):
    """
    Set the random seed for reproducibility.
    
    Args:
        seed (int): The seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Config and Target

In [2]:
RUN_ID = "experiment_a_test"
REGIME = "stationary"
REPLICATE = 1
T = 1000
SEED = 42
LOSS_NAME = "logistic"
MODEL_NAME = "memorypair"
LBFGS_MEM = 10

# Setup and Imports

In [3]:
# Import required modules
import sys
import os

# Add code path for imports
os.chdir(os.path.dirname(os.getcwd()))

from config import Config
from runner import ExperimentRunner

from agents.grid_runner import load_grid, generate_combinations, create_grid_id, run_parameter_combination


In [ ]:
import os, json
import yaml

GRID_FILE = "grids.yaml"
BASE_OUT = "results/grid_2025_08_15"   # date-stamp it
OUTPUT_GRANULARITY = "event"           # Experiment A wants per-event for the money plot
SEEDS = list(range(5))                 # match your CLI default

# Load grid config from YAML
with open(GRID_FILE, "r") as f:
    grid_config = yaml.safe_load(f)


combos = generate_combinations(grid_config)

# select 60 combinations randomly 
cases = random.sample(combos, 10)

print(f"{len(combos)} grid cells")
print(f"sampled {len(cases)} cells to run")

# (Optional) dry-run preview
for i, p in enumerate(cases[:5]):
    print(f"{i+1:2d}. {create_grid_id(p)} → {p}")

# Make output dirs
os.makedirs(os.path.join(BASE_OUT, "sweep"), exist_ok=True)

# Run the sweep using ExperimentRunner
all_csv = []
for i, params in enumerate(cases):
    print(f"\n=== Running cell {i+1}/{len(combos)}: {create_grid_id(params)} ===")
    csvs = run_parameter_combination(
        params=params,
        seeds=SEEDS,
        base_out_dir=BASE_OUT,
        output_granularity=OUTPUT_GRANULARITY,
        parallel=1,           # bump if your runner supports safe parallelism in-notebook
    )
    all_csv.extend(csvs)

# Persist a manifest like the CLI does (handy for analysis notebooks)
manifest = { create_grid_id(p): p for p in combos }
with open(os.path.join(BASE_OUT, "sweep", "manifest.json"), "w") as f:
    json.dump(manifest, f, indent=2)

print("\nDone. CSVs:", len(all_csv))


## Reading and Transforming the Data 

The data is now in a series of files in the results folder. This section will read in those datapoints and output a fully formatted dataframe of a random sample of the datapoints.

Analysis can be performed in batches.

In [5]:
# print a count of all directories in the results directory
dir_list = []
results_dir = "results/grid_2025_08_15/"
for r, d, files in os.walk(results_dir):
    for dirname in d:
        dir_list.append(os.path.join(r, dirname))
print(f"Number of directories in '{results_dir}': {len(dir_list)}")
print(dir_list)

Number of directories in 'results/grid_2025_08_15/': 27
['results/grid_2025_08_15/sweep', 'results/grid_2025_08_15/sweep/gamma_1.0-split_0.5_q0.95_k10_default_eps1.0_cmpdynamic_orcoff_prot_ang0.01_dr0.001_fs1_h3ebe6677', 'results/grid_2025_08_15/sweep/gamma_1.0-split_0.5_q0.95_k10_default_eps1.0_cmpdynamic_orcoff_prot_ang0.01_dr0.001_fs1_h303e9151', 'results/grid_2025_08_15/sweep/gamma_1.0-split_0.5_q0.95_k10_default_eps1.0_cmpdynamic_orcoff_prot_ang0.01_dr0.001_fs1_hd0eaf007', 'results/grid_2025_08_15/sweep/gamma_1.0-split_0.5_q0.95_k10_default_eps1.0_cmpdynamic_orcoff_prot_ang0.01_dr0.001_fs1_h6cf5bec1', 'results/grid_2025_08_15/sweep/gamma_1.0-split_0.5_q0.95_k10_default_eps1.0_cmpdynamic_orcoff_prot_ang0.01_dr0.001_fs1_h881e0dba', 'results/grid_2025_08_15/sweep/gamma_1.0-split_0.5_q0.95_k10_default_eps1.0_cmpdynamic_orcoff_prot_ang0.01_dr0.001_fs1_hd9cc12a7', 'results/grid_2025_08_15/sweep/gamma_1.0-split_0.5_q0.95_k10_default_eps1.0_cmpdynamic_orcoff_prot_ang0.01_dr0.001_fs1_h97ad

In [6]:
# go up a single level in the current directory
sys.path.append(os.path.dirname(os.getcwd()))

data = []
for d in dir_list:
    # ingest all event csv files
    for f in os.listdir(d):
        if f.endswith('events.csv'):
            df = pd.read_csv(os.path.join(d, f))
            data.append(df)

# concatenate all event dataframes
if data:
    data = pd.concat(data, ignore_index=True)
else:
    data = pd.DataFrame()


In [7]:
data.columns

Index(['C_hat', 'D_hat', 'G_hat', 'N_gamma', 'N_star_theory', 'P_T', 'P_T_est',
       'S_scalar', 'acc', 'accountant', 'avg_regret', 'avg_regret_with_noise',
       'base_eta_t', 'c_hat', 'comparator_type', 'cum_regret',
       'cum_regret_with_noise', 'd_norm', 'delta_step_theory', 'delta_total',
       'drift_boost_remaining', 'drift_flag', 'eps_remaining', 'eps_spent',
       'eps_step_theory', 'eta_t', 'event', 'event_id', 'event_type',
       'lambda_est', 'lambda_raw', 'm_capacity', 'm_used', 'noise_regret_cum',
       'noise_regret_increment', 'op', 'pair_admitted', 'pair_damped',
       'path_regret_increment', 'regret', 'regret_dynamic', 'regret_increment',
       'regret_path_term', 'regret_static_term', 'rho_remaining', 'rho_spent',
       'rho_step', 'sample_id', 'sc_active', 'sc_stable', 'segment_id',
       'sens_delete', 'sigma_step', 'sigma_step_theory',
       'static_regret_increment', 'x_norm', 'seed', 'grid_id', 'gamma_bar',
       'gamma_split', 'N_star_live', 'm_

In [9]:
data.head()


,C_hat,D_hat,G_hat,N_gamma,N_star_theory,P_T,P_T_est,S_scalar,acc,accountant,...,m_theory_live,blocked_reason,path_type,rotate_angle,drift_rate,feature_scale,w_scale,fix_w_norm,noise_std,version https://git-lfs.github.com/spec/v1
0,1.0,0.455604,29.975769,NaN,NaN,0.0,0.0,8.460643,0.632220,eps_delta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.455604,29.975769,NaN,NaN,0.0,0.0,239.559837,3.604286,eps_delta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,0.455604,29.975769,NaN,NaN,0.0,0.0,1447.181477,8.371785,eps_delta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.455604,29.975769,NaN,NaN,0.0,0.0,2486.203561,6.427817,eps_delta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,0.455604,29.975769,NaN,NaN,0.0,0.0,2614.986210,2.778743,eps_delta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
